I've been able to trace the bug back to Aaron's deconvolve function. Given two seemingly normal inputs, it will sometimes return all NaN's. This notebook is to figure out why. 

In [96]:
outputDir = '/u/ki/swmclau2/des/TestOut/'
dataDir = '/nfs/slac/g/ki/ki18/des/cpd/psfex_catalogs/SVA1_FINALCUT/psfcat/00180000/00180743/'

In [97]:
import numpy as np
opt_stamps = np.load(outputDir + '00180743/180743_opt_test.npy')

In [98]:
from glob import glob
from astropy.io import fits
meta_hdulist = []

filenames = glob(dataDir+'*_selpsfcat.fits')

for fname in filenames:
    meta_hdulist.append(fits.open(fname))

In [99]:
vig_idx=0
vignettes = np.zeros((opt_stamps.shape[0], 32,32))
hdu_lengths = np.zeros((62,))
for ccd_num, hdulist in enumerate(meta_hdulist):
    #TODO Turn sliced off pixels into background estimate

    list_len = hdulist[2].data.shape[0]

    sliced_vig  = hdulist[2].data['VIGNET'][:, 15:47, 15:47] #slice to same size as stamps
    sliced_vig[sliced_vig<-1000] = 0 #set really negative values to 0; it's a mask
    sliced_vig = sliced_vig/sliced_vig.sum((1,2))[:, None, None] #normalize
    vignettes[vig_idx:vig_idx+list_len] = sliced_vig
    vig_idx+=list_len
    vig_shape = hdulist[2].data['VIGNET'][0].shape
    #print 'CCD: %d\tVignette Shape:(%d, %d)'%(ccd_num+1, vig_shape[0], vig_shape[1] )

    hdu_lengths[ccd_num] = list_len

In [ ]:
import numpy as np
import scipy
import numpy.lib.index_tricks as itricks
import pdb
from WavefrontPSF.psf_evaluator import Moment_Evaluator
#from scipy.signal import convolve2d as convolve

def convolve(A, B):
    """ Performs a convolution of two 2D arrays """
    C = np.fft.ifft2(np.fft.fft2(A) * np.fft.fft2(B))
    C = np.fft.fftshift(C)
    C = C / np.sum(C)
    return np.real(C)

def convolveStar(A, B):
    """ Performs a convolution of two 2D arrays, but take the complex conjugate of B """
    C = np.fft.ifft2(np.fft.fft2(A) * np.conjugate(np.fft.fft2(B)))
    C = np.fft.fftshift(C)
    C = C / np.sum(C)
    return np.real(C)

def calcChi2(PSF,psi_r,phi_tilde,beta,mu0):
    """ Calculate chi2 between PSF convolved with restored image and measured image
    """
    imageC = beta * convolve(PSF,psi_r)
    diffImage = phi_tilde-imageC
    varianceImage = phi_tilde + mu0
    chi2 = np.sum(diffImage*diffImage/varianceImage)
    return chi2

def makeGaussian(shape,Mxx,Myy,Mxy):
    """ Return a 2-d Gaussian function, centered at 0, with desired 2-nd order moments
    """
    ny = shape[0]
    nx = shape[1]
    ylo = -ny/2. + 0.5
    yhi = ny/2. - 0.5
    xlo = -nx/2. + 0.5 
    xhi = nx/2. - 0.5
    yArr,xArr = itricks.mgrid[ylo:yhi:1j*ny,xlo:xhi:1j*nx]
    rho = Mxy/np.sqrt(Mxx*Myy)

    gaussian = np.exp( -((yArr*yArr)/Myy + (xArr*xArr)/Mxx - 2.*rho*xArr*yArr/np.sqrt(Mxx*Myy))/(2.*(1-rho*rho))  )
    #if np.any(np.isnan(gaussian)):
        #print 'Failed on making initial guess.'
        
    return gaussian

def makeMask(image,sigma,nsigma=3.):
    """ build a mask from the noisy Image
    """
    mask = np.where(image>nsigma*sigma,1.,0.)

    # use working copy
    maskcopy = mask.copy()

    # mask edge
    maskcopy[0,:] = 0.
    maskcopy[-1,:] = 0.
    maskcopy[:,0] = 0.
    maskcopy[:,-1] = 0.

    # demand that pixels have 3 neighbors also above 3sigma
    shape = mask.shape
    for j in range(1,shape[0]-1):
        for i in range(1,shape[1]-1):
            if mask[j,i]==1:
                # check 8 neighbors
                nNeigh = mask[j+1,i-1] + mask[j+1,i] + mask[j+1,i+1] + mask[j,i-1] + mask[j,i+1] + mask[j-1,i-1] + mask[j-1,i] + mask[j-1,i+1]
                if nNeigh<3:
                    maskcopy[j,i] = 0.

    # fill return array
    mask = maskcopy.copy()
    return mask

def deconvolve(PSF,phi_tilde,psi_0=None,mask=None,mu0=0.0,niterations=10,convergence=-1,chi2Level=0.0,extra=False):
    """ Implementation of the Richardson-Lucy deconvolution algorithm.
    Notation follows Lucy 1974, Eqn 15 and 14.  Add  noise term following
    Snyder et al 1993.
    Arguments
    ---------
    PSF          known Point Spread Function
    phi_tilde    measured object
    psi_0        starting guess for deconvolution
    mask         =0 for bins where we know that recovered image has no flux 
    mu0          background noise estimate
    """

    # normalize PSF
    PSF = PSF / np.sum(PSF)

    # if no initial guess, make one from 2nd moments of input image - PSF
    if psi_0 == None:
        # calculate Moments of psi_tilde and PSF, subtract and

        # calculate moments
        evaluator = Moment_Evaluator()

        # try a better starting guess - based on our knowledge of the PSF
        image_moments = evaluator(phi_tilde)
        PSF_moments = evaluator(PSF)

        #TODO what to do if makeGaussian throws an error?
        # subtract 2nd order moments in quadrature, use an object with the difference
        Mxx = image_moments['Mxx'][0] - PSF_moments['Mxx'][0]
        Myy = image_moments['Myy'][0] - PSF_moments['Myy'][0]
        Mxy = image_moments['Mxy'][0] - PSF_moments['Mxy'][0]

        psi_r = makeGaussian(phi_tilde.shape,Mxx,Myy,Mxy)
        
        if np.any(np.logical_or( np.isinf(psi_r), np.isnan(psi_r))):
            psi_r = phi_tilde #trying this out; otherwise we'll just have to raise errors/hell
        
    else:
        # initial guess
        psi_r = np.abs(psi_0)

    # mask starting guess
    if mask != None:    
        psi_r = psi_r * mask
        
    # normalize starting guess
    psi_r = psi_r / np.sum(psi_r)
    
    if np.any(np.isnan(psi_r)):
        raise RuntimeWarning("NaN in initial guess, skip this value. ")

    # mask image too
    if mask != None:
        phi_tilde = phi_tilde * mask
        
    # find normalization for measured image
    beta = np.sum(phi_tilde)
        
        
    # now iterate, either until convergence reached or fixed number of iterations are done
    psiByIter = []
    diffByIter = []
    chi2ByIter = []
    iteration = 0
    continueTheLoop = True
    while continueTheLoop: 

            
        # calculate next approximation to psi
        phi_r = beta*convolve(psi_r,PSF) + mu0
        psi_rplus1 = psi_r * convolveStar((phi_tilde+mu0)/phi_r,PSF)
    
        # mask the next iteration
        if mask != None:
            psi_rplus1 = psi_rplus1 * mask
        
        # normalize it
        psi_rplus1 = psi_rplus1 / np.sum(psi_rplus1)

        # check for convergence if desired
        #Why are the psiByIter appends inside the convergence test?
        if convergence>0:
            # compare psi_r and psi_rplus1
            psiByIter.append(psi_rplus1)
            diff = np.sum(np.abs(psi_rplus1 - psi_r))
            diffByIter.append(diff)
            if diff<convergence:
                continueTheLoop = False

        # also calculate how close to a solution we are
        chi2 = calcChi2(PSF,psi_rplus1,phi_tilde,beta,mu0)
        chi2ByIter.append(chi2)
        if chi2<chi2Level:
            continueTheLoop = False
        
        # check for Chi2 level
                

        # always check number of iterations
        if iteration==niterations-1:
            continueTheLoop = False

        # save for next iteration
        iteration+=1     
        psi_r = np.array(psi_rplus1)  # does a deepcopy


    #TODO rescale deconv by flux

    # we are done!
    if extra:
        return psi_rplus1,diffByIter,psiByIter,chi2ByIter
    else:
        return psi_rplus1        
 

In [ ]:
from itertools import izip
from matplotlib import pyplot as plt
%matplotlib inline
resid_list = []
bad_stars = set() #keep idx's of bad stars
for idx, (optpsf, vignette) in enumerate(izip(opt_stamps, vignettes)):
    #resid_small,diffs,psiByIter,chi2ByIter = deconvolve(optpsf,vignette,psi_0=None,mask=None,mu0=6e3,convergence=1e-3,chi2Level=0.,niterations=50, extra= True)
    resid = np.zeros((63,63))
    try:
        resid_small = deconvolve(optpsf,vignette,psi_0=None,mask=None,mu0=6e3,convergence=1e-3,chi2Level=0.,niterations=50, extra= False)


        resid[15:47, 15:47] = resid_small
        
    except RuntimeWarning: #Some will fail
        bad_stars.add(idx)
        #TODO check how this mask works
        resid = np.ones((63,63))*-9999 #forcing a mask

        cp_idx = idx#still need this, since we're going to keep iterating.
        for ccd_num, hdu_len in enumerate( hdu_lengths) :
            if hdu_len > cp_idx:
                print 'Deconvolve failed on CCD %d Image %d'%(ccd_num+1, cp_idx)
                break
            else:
                cp_idx-=hdu_len

    resid_list.append(resid)
print len(bad_stars)